In [1]:
import os

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\ML\\Projects\\NLP\\SummarizeText'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    data_path: Path
    metrics_file_name: Path
    tokenizer_path: Path

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager():
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_ModelEvaluationConfig(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path= config.model_path,
            data_path = config.data_path,
            metrics_file_name = config.metrics_file_name,
            tokenizer_path = config.tokenizer_path
        )
        

        return model_evaluation_config

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset
from evaluate import load as load_metric
from datasets import load_from_disk
import torch

ModuleNotFoundError: No module named 'torch'

In [16]:
import pandas as pd
import tqdm


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self,list_of_elements,batch_size):
        """split the data into small batches that we can process simultaneousl 
        yield successive batch-sized chunks from list_of_elemets """
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]

    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,batch_size= 16,device = 'cuda' if torch.cuda.is_available() else 'cpu',column_text='article',
                      column_summary='highlights'):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))

        for article_batch,target_batch in tqdm(
            zip(article_batches,target_batches),total=len(article_batches)):

            inputs = tokenizer(article_batch,max_length=1024,truncation=True,padding='max_length',return_tensors='pt')

            summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                        attention_mask=inputs['attention_mask'].to(device),length_penalty=0.8,
                                        num_beams=8,max_length=128)

            decoded_summaries = [tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True)
                                for s in summaries]

            decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries,references=target_batch)

        score = metric.compute()

        return score

    def evaluate(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)

        dataset_samsum = load_from_disk(self.config.data_path)

        rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

        rouge_metric = load_metric("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset_samsum['test'],rouge_metric,model_pegasus,tokenizer,
                                    batch_size=2,column_text='dialogue',
                                    column_summary='summary')
        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=['peagasus'])
        df.to_csv(self.config.metrics_file_name, index=False)

NameError: name 'torch' is not defined

In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_ModelEvaluationConfig()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2025-07-19 15:23:23,411: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2025-07-19 15:23:23,415: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-07-19 15:23:23,417: INFO: common: Created directory at: artifacts:]
[2025-07-19 15:23:23,419: INFO: common: Created directory at: artifacts/model_evaluation:]


NameError: name 'ModelEvaluation' is not defined